## 필요한 모듈 import

In [ ]:
import win32com.client
import pandas as pd
import numpy as np
import time
import multiprocessing
from datetime import datetime,timedelta

In [ ]:
YEAR=2023
MONTH=6
DAY=26

## 대신증권 CyberPlus와 연결위한 32비트 환경인지 확인

In [ ]:
import platform
print(platform.architecture())

('32bit', 'WindowsPE')


## cpbosplus와 연결

In [ ]:
instCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
print(instCpCybos.IsConnect)

1


## 대신증권 CYBOS PLUS에서 차트 정보 가져오는 함수

In [ ]:
def dashinChart(code, startdate, enddate):
    import win32com.client

    # Create object
    instStockChart = win32com.client.Dispatch("CpSysDib.StockChart")

    # SetInputValue
    instStockChart.SetInputValue(0, code)  #종목코드
    instStockChart.SetInputValue(1, ord('1'))   #1: 기간 2: 개수
    # 필드 2,3은 필드1에서 1.기간으로 요청시사용
    instStockChart.SetInputValue(2, enddate) #수집 종료 날짜
    instStockChart.SetInputValue(3, startdate) #수집 시작 날짜
    # 필드 4는 필드1에서 2.개수로 요청시 사용
    # instStockChart.SetInputValue(4, 2000)
    #5 필드
    instStockChart.SetInputValue(5, (0,1,2,3,4,5,8,9)) #필드값: 날짜,시간,시고저종,거래량,거래대금,상장주식수,시가총액
    instStockChart.SetInputValue(6, ord('m'))  # m 분단위
    instStockChart.SetInputValue(7,1)          # 주기 default-1
    # instStockChart.SetInputValue(8,ord('0')) # 갭보정 default-0 무보정
    instStockChart.SetInputValue(9, ord('1'))  # 1 (수정주가로 요청)
    instStockChart.SetInputValue(10, ord('3'))  # 3 (시간외 거래 모두제외)

    # BlockRequest
    instStockChart.BlockRequest()

    return instStockChart

## API로 가져온 결과 dataframe으로 변환

In [ ]:
def make_df(instStockChartList):
    dataframe=[]
    for instStockChart in instStockChartList:
        # GetHeaderValue
        numData =instStockChart.GetHeaderValue(3)  #데이터 개수
        numField =instStockChart.GetHeaderValue(1) #컬럼수
        save=[]
        for idx in range(numData):
            tmp=[]
            for col in range(numField):
                tmp.append(instStockChart.GetDataValue(col, idx))
            # print(tmp)
            save.append(tmp)

        columns_name=['날짜','시간','시가','고가','저가','종가','거래량','거래대금']
        df=pd.DataFrame(save,columns=columns_name)
        dataframe.append(df)
    # ===============================================================
    total_df=pd.concat(dataframe, ignore_index=True)
    total_df.sort_values(['날짜','시간'],inplace=True, ignore_index=True)

    return total_df

## 차트 데이터 .csv로 저장하기

In [ ]:
def save_csv_sd_ed(code,dataframe,sd=None,ed=None):
    csv_name=f'./save_data/{code}_{sd}_{ed}.csv' #./저장 디레토리/저장 파일명.csv
    dataframe.to_csv(csv_name, index=False)


### 실행 - 데이터프레임 변환 - csv로 저장

In [ ]:
start = time.time()
try:
    sd=datetime(YEAR,MONTH,DAY)
    ed=sd

    code='A247540'  #에코프로비엠
    stockchartlist=[]
    for i in range(5):
        ed=sd+timedelta(4)
        instStockChart=dashinChart(code=code,startdate=sd.strftime('%Y%m%d'),enddate=ed.strftime('%Y%m%d'))

        code=instStockChart.GetHeaderValue(0) #종목코드
        numData =instStockChart.GetHeaderValue(3)  #데이터 개수
        numField =instStockChart.GetHeaderValue(1) #컬럼수
        print(f'{sd}_{ed}, {code}, {numData}, {numField}')

        stockchartlist.append(instStockChart)
        sd=sd+timedelta(7)
        time.sleep(0.3)
        # ==================================================
    dataframe=make_df(stockchartlist) #수집된 기간들을 하나의 데이터 프레임으로 만들어주는 함수
    save_csv_sd_ed(code,dataframe,'20230626',ed.strftime('%Y%m%d')) #데이터 프레임을 csv파일로 저장하는 함수
except:
    print(print(f'{sd}_{ed}, {code}'))
end = time.time()
print("수행시간: %f 초" % (end - start))
# ===================================================


2023-06-26 00:00:00_2023-06-30 00:00:00, A247540, 1905, 8
2023-07-03 00:00:00_2023-07-07 00:00:00, A247540, 1905, 8
2023-07-10 00:00:00_2023-07-14 00:00:00, A247540, 1905, 8
2023-07-17 00:00:00_2023-07-21 00:00:00, A247540, 1903, 8
2023-07-24 00:00:00_2023-07-28 00:00:00, A247540, 1891, 8
수행시간: 2.176213 초


In [ ]:
dataframe

,날짜,시간,시가,고가,저가,종가,거래량,거래대금
0,20230626,901,263500,264500,263500,264000,4694,1237525000
1,20230626,902,263500,264500,263000,263000,3729,982223000
2,20230626,903,263500,264000,263000,264000,1690,445352000
3,20230626,904,264000,264000,263500,264000,3733,985080000
4,20230626,905,264000,264000,263000,263000,2936,773809000
...,...,...,...,...,...,...,...,...
9504,20230728,1517,406500,407000,406000,406000,6315,2565773000
9505,20230728,1518,406000,406500,405500,406000,5616,2280442000
9506,20230728,1519,405500,407500,405500,407000,12317,5006354000
9507,20230728,1520,407000,407500,406500,407500,8598,3500054000
